In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfJohXn4vNtDTppJwXXX1CL5Z8TBlkFO7kw81SxmpRn3BXlVs5iwyb9V1Vk



Successfully saved authorization token.


In [45]:
# Define the corners of the large ROI
top_left = (70, 45)
bottom_right = (110, 23)

# Define the grid size
cols = 5  # Number of columns
rows = 4  # Number of rows

# Calculate the step size for each grid cell
width_step = (bottom_right[0] - top_left[0]) / cols
height_step = (top_left[1] - bottom_right[1]) / rows


In [46]:
def create_grid(top_left, bottom_right, width_step, height_step, cols, rows):
    grid = []
    for i in range(cols):
        for j in range(rows):
            # Calculate corners of the cell
            left = top_left[0] + i * width_step
            right = left + width_step
            top = top_left[1] - j * height_step
            bottom = top - height_step

            # Create a rectangle for this cell
            cell = ee.Geometry.Rectangle([left, bottom, right, top])
            grid.append(cell)
    return grid

grid = create_grid(top_left, bottom_right, width_step, height_step, cols, rows)
grid[0]

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              70.0,
              45.0
            ],
            [
              70.0,
              39.5
            ],
            [
              78.0,
              39.5
            ],
            [
              78.0,
              45.0
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [47]:
gsw_monthly = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory')
yearlist = ee.List.sequence(2012, 2013)
reducer = ee.Reducer.max()

def create_yearly_composite(year):
    year = ee.Number(year)
    yearCol = gsw_monthly.filter(ee.Filter.calendarRange(year, year, 'year'))
    yearMax = yearCol.reduce(reducer)

    imgList = yearCol.aggregate_array('system:index')
    n_img = imgList.size()
    nBands = yearMax.bandNames().size()

    return yearMax.set({
        'year': year,
        'image_list': imgList,
        'n_bands': nBands,
        'n_img': n_img,
        'method': 'max value',
        'system:time_start': year
    })

yearCompList = yearlist.map(create_yearly_composite)
annual_max_water = ee.ImageCollection.fromImages(yearCompList).filter(ee.Filter.gt('n_bands', 0))
annual_list = annual_max_water.toList(annual_max_water.size())

# Function to calculate the difference between two images
def difference(img1, img2):
    def mask_only_water(img):
        return img.updateMask(img.eq(2)).multiply(0).add(1).unmask(0)

    img1_mask = mask_only_water(img1)
    img2_mask = mask_only_water(img2)

    diff = img2_mask.subtract(img1_mask).copyProperties(img1_mask, ['max value', 'n_img', 'system:time_start', 'year'])
    img_diff = ee.Image(diff)
    return img_diff

year1L = 2012
year2L = 2013

imgtest1 = ee.Image(annual_list.get(year1L - 2012))
# print(imgtest1)
imgtest2 = ee.Image(annual_list.get(year2L - 2012))

diff = difference(imgtest1, imgtest2)

In [48]:
# # def create_subgrid(region, dx, dy):
# #     grid = []
# #     start = region.coordinates().get(0).get(0)
# #     end = region.coordinates().get(0).get(2)

# #     lon_start = start.get(0)
# #     lat_start = start.get(1)
# #     lon_end = end.get(0)
# #     lat_end = end.get(1)

# #     current_lon = lon_start
# #     while current_lon.add(dx).getInfo() <= lon_end.getInfo():
# #         current_lat = lat_start
# #         while current_lat.add(dy).getInfo() <= lat_end.getInfo():
# #             square = ee.Geometry.Rectangle([current_lon.getInfo(), current_lat.getInfo(), current_lon.add(dx).getInfo(), current_lat.add(dy).getInfo()])
# #             grid.append(square)
# #             current_lat = current_lat.add(dy)
# #         current_lon = current_lon.add(dx)

# #     return grid

# def create_subgrid(region, dx, dy):
#     grid = []
    
#     # Convert the region to a list of coordinates
#     # coords = ee.List(region.coordinates().get(0))
#     coords = ee.List(region.coordinates())
    
#     # Extract the minimum and maximum coordinates
#     lon_min = ee.Number(coords.get(0).get(0))
#     lat_min = ee.Number(coords.get(0).get(1))
#     lon_max = ee.Number(coords.get(2).get(0))
#     lat_max = ee.Number(coords.get(2).get(1))

#     # Create a sequence for longitude and latitude
#     lons = ee.List.sequence(lon_min, lon_max, dx)
#     lats = ee.List.sequence(lat_min, lat_max, dy)

#     # Nested function to iterate over the coordinates
#     def create_rects(lon, acc):
#         def create_rect(lat):
#             rect = ee.Geometry.Rectangle([lon, lat, lon.add(dx), lat.add(dy)])
#             return ee.Feature(rect)
        
#         new_rects = lats.map(create_rect)
#         return ee.List(acc).cat(new_rects)

#     # Create the grid
#     grid = lons.iterate(create_rects, ee.List([]))

#     return ee.FeatureCollection(ee.List(grid))



In [59]:
# ee.Number(grid[0].coordinates().get(9))

In [49]:
# # Define the size of each grid cell (9 km x 9 km)
# dx = ee.Number(9 / 111.113)  # Approximate conversion of longitude degrees to kilometers at the equator
# dy = ee.Number(9 / 111.113)  # Approximate conversion of latitude degrees to kilometers

# # Generate the grid
# subgrid = create_subgrid(region=ee.Geometry.Rectangle([70.0, 39.5, 78.0, 45.0]), dx=dx, dy=dy)


AttributeError: 'ComputedObject' object has no attribute 'get'

In [41]:
# type(grid)

list

In [40]:
# ee.Geometry.Rectangle(grid[0])

EEException: A mapped function's arguments cannot be used in client-side operations

In [5]:
# # roi = ee.Geometry.Rectable([])
# summed_values = diff.reduceRegion(
#     reducer=ee.Reducer.sum()
#     ,geometry=cell
#     ,scale=9000
#     ,maxPixels=1e13
# )

# print(summed_values.getInfo())
# type(ee.Image(summed_values))
# diff_reduc = diff.reduceResolution(reducer=ee.Reducer.sum(), maxPixels=65535)
# diff_reduc = diff_reduc.reproject(crs=diff.projection().crs(), scale=9000)
# diff_reduc = diff.float()

In [6]:
# type(cell)

# diff.reduceRegion(/.reducer=ee.Reducer.sum(), scale=9000)

In [7]:
era5_land_temp = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('temperature_2m')
era5_land_precip = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('total_precipitation')

filtered_temp = era5_land_temp.filterDate('2012-01-01', '2013-12-31')
filtered_precip = era5_land_precip.filterDate('2012-01-01', '2013-12-31')

In [8]:
cell = grid[0]

temp_image = filtered_temp.filter(ee.Filter.calendarRange(2012, 2012, 'year')).mean().clip(cell)

precip_image = filtered_precip.filter(ee.Filter.calendarRange(2012, 2012, 'year')).sum().clip(cell)

# gsw_image = ee.ImageCollection(diff).sum().clip(cell)
gsw_image = ee.ImageCollection(ee.Image(diff))#.clip(cell)

In [20]:
# diff_reduc = diff.reduceResolution(reducer=ee.Reducer.sum(), maxPixels=1024)
# diff_reduc = diff_reduc.reproject(crs=precip_image.projection().crs(), scale=9000)
# diff_reduc = diff.float()
# Image.reduceRegion(reducer, geometry, scale, crs, crsTransform, bestEffort, maxPixels, tileScale)
diffr = diff.reduceRegion(reducer=ee.Reducer.sum()
                  ,geometry=grid[0]
                  ,scale=9000
                  ,crs=precip_image.projection().crs()
                 )
diffr

In [22]:
type(era5_land_temp)

ee.imagecollection.ImageCollection

In [21]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.Map
folium.Map.add_ee_layer = add_ee_layer

# Create a Folium map at a specific location
my_map = folium.Map(location=[20, 0], zoom_start=3)

viz_params = {'palette':['green','yellow','cyan']}

# Add the Earth Engine image as a layer
my_map.add_ee_layer(era5_land_temp, viz_params, 'Landsat 8 Image')

# Add a LayerControl widget
my_map.add_child(folium.LayerControl())

# Display the map
my_map






EEException: Image.visualize, argument 'image': Invalid type.
Expected type: Image<unknown bands>.
Actual type: ImageCollection.

In [14]:
def export_to_gdrive(img, name, region):
    task = ee.batch.Export.image.toDrive(**{
        'image':img
        ,'description': name
        ,'folder':'EarthEngineData'
        ,'scale':9000
        ,'region':region
        ,'fileFormat':'GeoTIFF'
    })
    task.start()
    print(f'task {name} sent to google...')

In [15]:
export_to_gdrive(img=temp_image, name='Temperature_2012_0', region=cell)
export_to_gdrive(img=precip_image, name='Precipitation_2012_0', region=cell)
export_to_gdrive(img=gsw_image, name='Difference_2012_0', region=cell)

task Temperature_2012_0 sent to google...
task Precipitation_2012_0 sent to google...
task Difference_2012_0 sent to google...


In [23]:
# import ee
# ee.Initialize()

# Load the ERA5-Land precipitation dataset
era5_land_precip = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('total_precipitation')


In [24]:
# Define the time period
start_date = '2020-01-01'
end_date = '2020-12-31'

# Sum the precipitation data over the defined period
precipitation_sum = era5_land_precip.filterDate(start_date, end_date).sum()


In [25]:
# Visualization parameters
vis_params = {
    'min': 0,
    'max': 3000,  # Adjust this based on your data's range
    'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}


In [29]:
import folium

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Create a Folium map
center = [20, 0]  # Adjust the center and zoom level to your area of interest
m = folium.Map(location=center, zoom_start=2)

# Add the Earth Engine precipitation layer
m.add_ee_layer(precipitation_sum.clip(grid[0]), vis_params, 'ERA5-Land Precipitation Sum')

# Add a layer control panel
folium.LayerControl().add_to(m)

# Display the map
m
